##**AML competition:**

**match the images in a query set with the images in a much larger set called gallery.**

**ResNet50 versions:**

this code has two parts: in the first part we check how the pre-trained model works, we extract the features and try to match images from the query to the gallery images. 
After this initial exploration, in the second and most important part, we fit the model, adding to it some layers, and produce a dictionary of matching images.

**FIRST PART**

Import the necessary libraries.

In [35]:
import numpy as np
from numpy.linalg import norm
from tqdm import tqdm, tqdm_notebook
import os
import cv2
import random
import time
import math
from scipy.spatial import distance
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
import utils

Use the colab GPU for faster execution of the code.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Connect colab to google drive, where the dataset with all the images is saved.

In [ ]:
!unzip -q /content/drive/MyDrive/unbalanced_dataset_2304.zip

In [4]:
data_path = '/content/unbalanced_dataset_2304/'

In [ ]:
class Dataset(object):
    def __init__(self, data_path):
        self.data_path = data_path
        assert os.path.exists(self.data_path), 'Insert a valid path!'

        # get class list
        self.data_classes = os.listdir(self.data_path)

        # init mapping dict
        self.data_mapping = {}

        # populate mapping dict, 
        #
        #{
        #   path_of_the_image: class 
        #}
        for c, c_name in enumerate(self.data_classes):
            temp_path = os.path.join(self.data_path, c_name)
            temp_images = os.listdir(temp_path)
            for i in temp_images:
                img_tmp = os.path.join(temp_path, i)

                if img_tmp.endswith('.jpg') or  img_tmp.endswith('.JPEG'):
                    if c_name == 'distractor':
                        self.data_mapping[img_tmp] = -1
                    else:
                        #self.data_mapping[img_tmp] = int(c_name)
                        self.data_mapping[img_tmp] = c_name

        print('Loaded {:d} from {:s} images'.format(len(self.data_mapping.keys()),
                                                    self.data_path))

    def get_data_paths(self):
        # returns a list of images paths and related classes
        images = []
        classes = []
        for img_path in self.data_mapping.keys():
            if img_path.endswith('.jpg') or img_path.endswith('.JPEG'):
                images.append(img_path)
                classes.append(self.data_mapping[img_path])
        return images, np.array(classes)


    def num_classes(self):
        # returns number of classes of the dataset
        return len(self.data_classes)



Define the model to use. Taking the pre-trained ResNet50 model from the keras library (https://keras.io/api/applications/). 

In [ ]:
#ResNet50 is a convolutional Neural Network with 50 layers, pretrained on more than a million images from ImageNet. It requires as input images of dimension 224x224.

model=ResNet50(weights='imagenet',
                         include_top=False, #does not include a fully-connected layer at the top of the network
                         input_shape=(224, 224, 3),
                        pooling='max') #global max pooling

Using keras, we firstly defined the ImageDataGenerator function, which specify all the modifications we want to make to our training images in order to add some noises and make it more challenging to associate query and gallery images.

In [7]:
datagen = ImageDataGenerator(
    featurewise_center=False, #set input mean to zero if True
    samplewise_center=True, #set each sample mean to zero
    featurewise_std_normalization=False, #divide inputs by sd of the dataset
    samplewise_std_normalization=True, #divide each input by its sd 
    rotation_range=20, #degree range for random rotation
    width_shift_range=0.3, #range as a fraction of total width within which to randomly translate pictures vertically 
    height_shift_range=0.3, #ranges as a fraction of total height within which to randomly translate pictures horizontally
    brightness_range=None,
    zoom_range=0.3,
    fill_mode="nearest",
    rescale=None, #all data are multiplied by specified value if given
    preprocessing_function=None, #preprocessing function to execute after the image is resized and augmented (input must be numpy tensor with rank 3)
    data_format=None 
)


In [8]:
#here we are using the class Dataset defined above, in the first part

validation_path = os.path.join(data_path, 'validation')
gallery_path = os.path.join(validation_path, 'gallery')
query_path = os.path.join(validation_path, 'query')
training_path = os.path.join(data_path, 'training') 

training_dataset = Dataset(data_path = training_path)
gallery_dataset = Dataset(data_path=gallery_path)
query_dataset = Dataset(data_path=query_path)


# get training data and classes
training_paths, training_classes = training_dataset.get_data_paths()

# get validation gallery and query data
gallery_paths, gallery_classes = gallery_dataset.get_data_paths()
query_paths, query_classes = query_dataset.get_data_paths()



Loaded 31690 from /content/unbalanced_dataset_2304/training images
Loaded 8630 from /content/unbalanced_dataset_2304/validation/gallery images
Loaded 2196 from /content/unbalanced_dataset_2304/validation/query images


Taking advantage of ImageDataGenerator, we add some noises to the images and we pre-process them using the specific pre-processing function of the keras-defined ResNet50.

In [9]:
def generators(preprocessing): 

    datagen = ImageDataGenerator(
        preprocessing_function = preprocessing,
        horizontal_flip = True, 
        validation_split = 0,
    )


    train_dataset = datagen.flow_from_directory(
        training_path,
        target_size = (224,224), 
        batch_size = 32,
        subset = 'training', 
    )

    return train_dataset

In [10]:
train_dataset = datagen.flow_from_directory(  
        training_path,
        target_size = (224,224), 
        batch_size = 32,
        subset = 'training',)

Found 33004 images belonging to 101 classes.


In [11]:
train_dataset = generators(preprocessing=preprocess_input) # CARLO: proper function for resnet50 

Found 33004 images belonging to 101 classes.


Now we create the model, starting with ResNet50 and adding to it some layes. Finally, we fit it on the training dataset.

In [13]:
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
for layer in resnet_model.layers:
    layer.trainable = False
x = Flatten()(resnet_model.output)
x = Dense(1000, activation='relu')(x)
preds = Dense(101, activation='softmax')(x)
full_model = Model(inputs=resnet_model.input, outputs=preds)
full_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [14]:
full_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#optimizer = Adam (learning_rate)


In [ ]:
history = full_model.fit(
    train_dataset,
    epochs=15,
    batch_size=32)



In [ ]:
SAVE_MODEL_WEIGHTS_PATH = '/content/drive/MyDrive/models/saved_weights/Resnet50SmallTraining.h5'
SAVE_MODEL_PATH = '/content/drive/MyDrive/models/saved_models/Resnet50SmallTraining.h5'

full_model.save_weights(SAVE_MODEL_WEIGHTS_PATH)
full_model.save(SAVE_MODEL_PATH)

## Challenge day
### Extract features and compute similarity

In [13]:
model = keras.models.load_model(SAVE_MODEL_PATH)


In [15]:
feat_extractor = Model(inputs=model.input, outputs=model.get_layer("dense").output)
feat_extractor.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [ ]:
!unzip -q /content/drive/MyDrive/challenge_test_data.zip

In [7]:
GALLERY_PATH = '/content/gallery'
QUERY_PATH = '/content/query'

In [ ]:
target_shape = (256, 256)

gallery = utils.Dataset(data_path=GALLERY_PATH).get_dataset()
query = utils.Dataset(data_path=QUERY_PATH).get_dataset()

## Extracting features for gallery and query

In [ ]:
import utils
gallery_features, gallery_urls, gallery_labels = utils.compute_features(gallery, feat_extractor)
query_features, query_urls, query_labels = utils.compute_features(query, feat_extractor)

results_eu = utils.compute_results(query_features, gallery_features, query_urls, gallery_urls)
results_cos = utils.compute_results(query_features, gallery_features, query_urls, gallery_urls, dist= 'cosine')

# Competition

In [ ]:
import requests
import json


def submit(results, url="http://tinyurl.com/IML2022"):
    res = json.dumps(results)
    response = requests.post(url, res)
    try:
        result = json.loads(response.text)
        print(f"accuracy is {result['results']}")
    except json.JSONDecodeError:
        print(f"ERROR: {response.text}")



mydata = dict()
mydata['groupname'] = "The Ostriches"

mydata["images"] = results_eu
submit(mydata)

mydata["images"] = results_cos
submit(mydata)

accuracy is [0.4153846153846154, 0.6153846153846154, 0.6615384615384615]
